<a href="https://colab.research.google.com/github/prakashradhakrish/Segmentation-using-UNet-Architecture/blob/master/Cityscape_extracting_pedestrian_Group_15_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cityscape pedestrian dataset**#
---
![alt text](https://drive.google.com/uc?id=1SEoA56Ndy2LvjCN8GL2tgXaoXus22zSp) 
![alt text](https://drive.google.com/uc?id=1xk1ElHqA9s58MXutjBa4ZX2GsaOWbbxI)
![alt text](https://drive.google.com/uc?id=1WRqjw0Kx5pb9QFgI4XF5OlCgWXykHHEw)
---



# **Initialisation**

In [ ]:
import numpy as np
import torch
import os
import fnmatch
import re
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as func
from torch import Tensor
import torchvision 
from torch.optim.lr_scheduler import StepLR
from IPython.display import HTML, display
from torchvision import transforms,datasets,utils
%matplotlib inline
from torch.utils.data import DataLoader, TensorDataset
import cv2

# **Dataset generation**

**Make sure the current directory is chosen**

In [ ]:
datadir_trainimage="E:/VE/4th quater/Computer Vision/Project/Notebooks/cityscape/leftImg8bit/val"
datadir_maskimage ="E:/VE/4th quater/Computer Vision/Project/Notebooks/cityscape/gtFine/val"


## Extracting subdirectories for training image and mask
sub_directory_img=[]
sub_directory_mask=[]

for x in os.walk(datadir_trainimage):
    sub_directory_img.append(x[0])
    print(x[0])
sub_directory_img.pop(0)

for x in os.walk(datadir_maskimage):
    sub_directory_mask.append(x[0])
sub_directory_mask.pop(0)

**Creating Masking images**

Extracting the masking information for pedestrian class


In [ ]:
maskingpath=[]
maskingfiles =[]
status_mask = 0

for sub in sub_directory_mask:
    path = "E:/VE/4th quater/Computer Vision/Project/Notebooks/cityscape/cityscape_400x400/gtFine/val"
    path=path+str('/')+str(os.path.basename(sub))
    maskingpath.append(str(path))
    try:
        os.makedirs(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    for i,img in enumerate(os.listdir(sub)):
        if fnmatch.fnmatch(img,'*labelIds.png'):
            img_array =cv2.imread(os.path.join(sub,img),cv2.IMREAD_GRAYSCALE)   
            new_array = cv2.resize(img_array,(400,400)) 
            mask = np.zeros(new_array.shape) # for binary class with pedestrian
            if (np.sum(new_array==24)) > 400:
                mask[np.where(new_array==24)]=255 
                os.chdir(path)
                cv2.imwrite(img, np.squeeze([mask]))
                maskingfiles.append(img)
                status_mask=status_mask+1      
    if status_mask%50 == 49:
        print("Completed images: " + str(status_mask))  

**Creating Target images**

In [ ]:
trainingpath=[]
trainingfiles =[]
status_img = 0

for i,sub in enumerate(sub_directory_img):
    path = "E:/VE/4th quater/Computer Vision/Project/Notebooks/cityscape/cityscape_400x400/leftImg8bit/val"
    path=path+str('/')+str(os.path.basename(sub))
    trainingpath.append(str(path))
    print(str(path))
    try:
        os.makedirs(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    for mask_sub in (os.listdir(maskingpath[i])):
        img = (re.findall('(.+)_gt', mask_sub)[0]) + str('_leftImg8bit.png')
        img_array =cv2.imread(os.path.join(sub,img),cv2.COLOR_RGB2GRAY)
        new_array = cv2.resize(img_array,(400,400))
        os.chdir(path)
        cv2.imwrite(img, np.squeeze([new_array]))
        trainingfiles.append(img)
        status_img=status_img+1     